In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
from torch import nn 

# Teaching a quadruped to walk  

Time to try out the learning algorithms that you just implemented on a more difficult problem. The WalkerEnv implements a quadruped robot kind-of thing, see for yourself. The goal is to move in the $x$ direction as fast and as far as possible. 

Your goal is to implement a class ``WalkerPolicy`` with function ``determine_actions()`` just like the StochasticPolicy we used earlier to control the pendulum. Below is a template of this class, but feel free to alter it however you want. The only important thing is the ``determine_actions()`` function! 

After you implement it, copy ``WalkerPolicy`` into a separate file ``WalkerPolicy.py`` that you will upload to BRUTE together with the (optional) learned weights in a zip file. How the policy is implemented is up to you! You are constrained to only the libraries we used so far though, such as torch, numpy etc..

You will get some free points just for uploading a working policy (irrelevant of the performance). Further 2 points will be awarded for successfully traversing a small distance in the x direction.

# Hints

There is no single easy way of doing this, but here are some suggestions on what you could try to improve your policy:

1) This problem is much more difficult, than balancing a pendulum. It is a good idea to use a bit larger network than for the pendulum policy.

2) You can also try to use a different optimizer, such as Adam and play with the hyperparameters.

3) Using a neural network to compute the normal distribution scale $\sigma$ can lead to too much randomness in the actions (i.e. exploration). You can use a fixed $\sigma$ instead, or replace it with a learnable ```torch.Parameter``` initialized to some small constant. Make sure, you run it through an exponential, or softplus function to ensure $\sigma$ is positive.

4) The exploration can also be reduced by penalizing the variance of the action distribution in an additional loss term. 

5) If you see some undesirable behaviour, you can tweak the reward function to penalize it. Even though the $x$ distance is all we care about, adding extra terms to the reward can help guide the learning process (This is known as reward shaping). Simply define a reward function mapping the state $s_{t+1}$ and action $a_t$ to a scalar reward $r_t$ and put it in the config dictionary under the key ```'reward_fcn'```. See the ```WalkerEnv``` class for the implementation of the default reward.

6) Using the normal distribution on a bounded action space can lead to certain problems caused by action clipping. This can be mitigated by using a different distribution, such as the Beta distribution. See the ```torch.distributions.beta``` module for more information. (Note that Beta distribution is defined on the interval [0,1] and works better with parameters $\alpha,\beta \geq 1$.)

In [2]:
from environment.WalkerEnv import WalkerEnv
from WalkerPolicy import WalkerPolicy
import solution

In [3]:
config = {'N': 1, 'vis': 1}
env = WalkerEnv(config)

Environment ready


In [4]:
obs = env.vector_reset()
env.render()

In [5]:
#obs[0, 0]  # this is the x coordinate of the robot, we want to maximize this
gamma = 0.95
epsilon = 0.2

def train(env, policy, optimizer, num_iterations=50):
    for iteration in range(num_iterations):
        # 1. Rollout
        observations, actions, rewards, log_probs = [], [], [], []
        done = False
        obs = env.reset()
        for i in range (512):
            obs = np.array(obs[0])  # Ensure obs is a numpy array
            obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)  # Convert to tensor and add batch dimension
            action, logp = policy.determine_actions(obs_tensor)
            action = action.squeeze(0).numpy()  # Remove batch dimension and convert to numpy
            next_obs, reward, done = env.step(action)
            observations.append(obs)  # Append the numpy array, not the tensor
            actions.append(action)
            rewards.append(reward)
            log_probs.append(logp)
            obs = next_obs

        # 2. Convert data to tensors
        print("obs:",observations)
        print("actions:",actions)
        print("rew:",rewards)
        print("logp:",log_probs)
        obs_tensor = torch.tensor(np.array(observations), dtype=torch.float32)
        act_tensor = torch.tensor(np.array(actions), dtype=torch.float32)
        rew_tensor = torch.tensor(np.array(rewards), dtype=torch.float32)
        logp_tensor = torch.stack(log_probs)

        # 3. Compute advantage or returns
        advantage_estimates = rew_tensor  # placeholder

        # 4. Policy loss
        loss = solution.ppo_loss(logp_tensor, advantage_estimates, gamma, epsilon)  # placeholder

        # 5. Optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 6. Logging
        if iteration % 10 == 0:
            print(f"Iteration {iteration}: Loss = {loss.item()}")

    env.close()

policy = WalkerPolicy()
optimizer = torch.optim.Adam(policy.parameters(), lr=1e-3)
train(env, policy, optimizer, num_iterations=50)

policy = WalkerPolicy()
optimizer = torch.optim.Adam(policy.parameters(), lr=1e-3)
train(env, policy, optimizer, num_iterations=50)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (512,) + inhomogeneous part.

In [6]:

for i in range(512):
    a = np.random.randn(1, 8)
    obs, reward = env.vector_step(a)
env.close()

[array([ 3.3581263e-04,  2.9005856e-05,  9.9801576e-01,  9.9999970e-01,
        6.1167817e-04, -3.9846014e-04, -1.2730312e-04,  3.3590704e-02,
        3.9027672e-02, -1.5660657e-02, -4.7554251e-02, -2.4656858e-02,
       -4.7604021e-02,  1.8170521e-02, -2.6822986e-02,  3.1504903e-02,
        3.8350192e-03, -2.0031334e-01,  1.1803192e-01, -7.3200844e-02,
       -1.6614715e-02,  3.2225707e+00,  3.7392371e+00, -1.5014125e+00,
       -4.5581317e+00, -2.3642735e+00, -4.5623550e+00,  1.7415758e+00,
       -2.5704877e+00], dtype=float32)]
[array([ 3.75884731e-04,  7.27171369e-04,  9.92011130e-01,  9.99996185e-01,
        2.32155761e-03, -1.27295800e-03,  8.15047359e-04,  9.17986706e-02,
        1.10954411e-01, -8.61470178e-02, -8.05000365e-02, -1.20283484e-01,
       -1.08464465e-01,  1.09699070e-01, -1.19786546e-01, -3.13481912e-02,
        6.22364692e-02, -4.00046796e-01,  2.28060052e-01, -9.76497158e-02,
        1.80095434e-01,  2.62625599e+00,  3.46464014e+00, -5.37797117e+00,
        1.0

[array([ 2.0975323e-02, -3.3230744e-03,  8.0034482e-01,  9.9942338e-01,
       -8.0458401e-03, -5.2162805e-03,  3.2572914e-02,  4.4075832e-01,
        1.0936306e+00, -5.3975999e-01,  1.1287131e-01, -1.4392424e-02,
       -3.6352187e-01, -6.4166164e-01, -1.7192401e-01, -1.1504725e-02,
       -6.4962752e-02, -1.9498187e+00, -2.2854507e-01,  5.2654557e-02,
       -2.9541960e-01, -3.7036768e-01, -1.2308283e+00,  1.5185997e+00,
        9.7048283e+00,  7.2679663e-01,  1.6753980e+00, -3.8895401e-01,
        1.5390946e+00], dtype=float32)]
[array([ 1.9794378e-02, -5.5748727e-03,  7.5951612e-01,  9.9951231e-01,
       -9.9845165e-03, -4.7689145e-03,  2.9200910e-02,  5.0530881e-01,
        1.0335757e+00, -4.4312114e-01,  2.5442484e-01, -7.0713788e-02,
       -3.5138914e-01, -6.3347012e-01, -1.6038093e-01, -1.1859161e-01,
       -1.5285596e-01, -2.1312618e+00, -1.8245877e-01,  3.7823956e-02,
       -2.7978218e-01,  4.4715466e+00, -4.6259904e+00,  7.8723321e+00,
        4.6626477e+00, -6.0926719e+